In [1]:
import gym, acme_gym
import numpy as np
import copy
from scipy import linalg, integrate
from tqdm import tqdm

In [2]:
def linearized_init(M, m, l, q1, q2, q3, q4, r):
    '''
    Adjusted for cart pole

    Parameters:
    ----------
    M, m: floats
        masses of the rickshaw and the present
    l   : float
        length of the rod
    q1, q2, q3, q4, r : floats
        relative weights of the position and velocity of the rickshaw,
        the angular displacement theta and the change in theta,
        and the control

    Return
    -------
    A : ndarray of shape (4,4)
    B : ndarray of shape (4,1)
    Q : ndarray of shape (4,4)
    R : ndarray of shape (1,1)
    '''
    g = 9.8
    a = 1./12
    A = np.array([[0,1,0,0],
                  [0,0,-3*m*g/(4*M+m),0],
                  [0,0,0,1],
                  [0,0,(9*m*g)/(8*l*M + 2*l*m) + 3*g/(2*l),0]])
    
    B = np.array([0,1./(M + m/4), 0, -3./(2*l*(M + m/4))])
    Q = np.diag([q1,q2,q3,q4])
    R = np.array([r])
    return A,B,Q,R

In [3]:
def cart(tv, X0, A, B, Q, R, P):
    '''
    adjusted for cart pole

    Parameters:
    ----------
    tv  : ndarray of time values, with shape (n+1,)
    X0  : Initial conditions on state variables
    A, Q: ndarrays of shape (4,4)
    B   : ndarray of shape (4,1)
    R   : ndarray of shape (1,1)
    P   : ndarray of shape (4,4)
    Returns
    -------
    Z : ndarray of shape (n+1,4), the state vector at each time
    U : ndarray of shape (n+1,), the control values
    '''
    def ode(z,t):
        return (A - np.outer(B,B.T@P)/R).dot(z)
    Z = integrate.odeint(ode, X0, tv)
    U = -np.dot(B,P@Z.T)/R
    return Z,U

In [4]:
def cartpole(environment, init_state, tol=1e-2, disp=False, N = 300):
    env = environment
    obs = init_state
    
    control = []
    
    '''
    This function will run the cartpole problem using the environment and initial conditions provided.
    Do NOT call env.reset().
    Run whatever system you desire to make sure the state values fall under the tolerance level.
    Convergence is considered reached once numpy.linalg.norm(obs[1:]) < tol, where we ignore the x position of the cart.
    You will need to return the sequence of controls that brings the cartpole system into vertical stability.
    Make sure to quit after N iterations, or convergence is reached, whichever occurs first.
    Remember you are being graded against other teams' step counts, so you want to end the system updates as soon as possible.
    
    Parameters:
        environment (CartPole object): The cartpole environment as described in gym and acme_gym.
        init_state (tuple): The initial state with [x, x', θ, θ'].
        tol (float): The tolerance to be reached before the cartpole problem is considered converged.
        disp (bool): If True, render the image.
        N (int): The max number of iterations.
    
    Returns:
        (list): A list of control values. These will be tested on the grader's end for convergence and time step assessment.
    
    '''
    #T = round(12/0.02)
    M,m,l = 1,.1,.5
    A,B,Q,R = linearized_init(M,m,l,0,1,1,1,.001)
    P = linalg.solve_continuous_are(A,B.reshape((4,1)),Q,R)
    for i in range(N):
        if linalg.norm(obs[1:]) < tol:
            env.close()
            break
            # Determine the step size based on our mode
        else:
            z, u = cart(np.arange(i*.02,N,.02), obs, A,B,Q,R,P)
            step = np.array([u[0]])
            control.append(u[0])

            # Step in designated direction and update the visual
            obs, reward, state, info = env.step(step)
        if disp:
            env.render()

    
    env.close()
    return control

In [5]:
tol = 1e-2
total = 200

render = True

num_steps = []

for i in tqdm(range(total)):
    env = gym.make("CartPoleContinuous-v0")
    observation = env.reset()
    env_copy = copy.deepcopy(env)
    
    # This is students' code
    control = cartpole(env_copy, observation, tol, render)
    
    for u in control:
        obs, reward, state, info = env.step(np.array([u]))
        
        if render:
            env.render()
    
    render = False
    
    if np.linalg.norm(obs[1:]) > tol:
        print("unsuccessful")
    
    num_steps.append(len(control))
    env.close()

print("Average number of steps after {} iterations: {}".format(total, np.mean(num_steps)))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [02:03<00:00,  1.61it/s]


Average number of steps after 200 iterations: 88.545
